In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')

In [28]:
### Season to test results

N = 2021

In [29]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(['season', 'round', 'podium', 'driver_points_from_race'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from_race.values)

In [30]:
X_train.columns

Index(['driver_standings_pos', 'constructor_standings_pos', 'starting_grid',
       'driver_points_last_3_races', 'driver_adrian_sutil',
       'driver_alexander_albon', 'driver_alexander_rossi',
       'driver_andre_lotterer', 'driver_antonio_giovinazzi',
       'driver_brendon_hartley',
       ...
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuka', 'circuit_id_villeneuve', 'circuit_id_yas_marina',
       'circuit_id_zandvoort'],
      dtype='object', length=108)

In [31]:
comparison_dict ={'model':[],
                  'criterion': [],
                  'max_features': [],
                  'max_depth': [],
                  'score': []
                  }

In [32]:
def score_regression(X_train, y_train, model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from_race'], axis=1)
        y_test = test.driver_points_from_race

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('predicted_points', ascending=False, inplace=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        correct_predictions += precision_score(prediction_df.actual_winner, prediction_df.predicted_winner)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        
    # return prediction score
    return correct_predictions / df[df.season == N]['round'].nunique()

In [33]:
# Random Forest Regressor

test = merged.copy()
test = test[['season', 'round', 'driver', 'starting_grid', 'podium']].query('season ==@N')

params={'criterion': ['squared_error'],
        'max_features': [0.8, 1.0],
        'max_depth': list(map(int, (np.linspace(5, 55, 26))))
        }

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:

            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            
            score = score_regression(X_train, y_train, model)

            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['criterion'].append(criterion)
            comparison_dict['max_features'].append(max_features)
            comparison_dict['max_depth'].append(max_depth)
            comparison_dict['score'].append(score)

In [35]:
comparison_df = pd.DataFrame(comparison_dict)

chosen_inputs = comparison_df.query('score > 0.5').sort_values('score', ascending=False)


chosen_inputs.to_csv(path+'parameters/rf_regressor.csv', index=False)

chosen_inputs.head()

,model,criterion,max_features,max_depth,score
30,random_forest_regressor,squared_error,1.0,13,0.590909
34,random_forest_regressor,squared_error,1.0,21,0.590909
47,random_forest_regressor,squared_error,1.0,47,0.590909
11,random_forest_regressor,squared_error,0.8,27,0.590909
44,random_forest_regressor,squared_error,1.0,41,0.590909
